In [4]:
import pandas as pd

csv_path = "../data/Stanford_Data.csv"
solar_df = pd.read_csv(csv_path, delimiter=";" , encoding="ANSI")
solar_df.head()

,Unnamed: 0,tile_count,solar_system_count,total_panel_area,fips,average_household_income,county,education_bachelor,education_college,education_doctoral,...,incentive_count_nonresidential,incentive_residential_state_level,incentive_nonresidential_state_level,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate
0,0,0,0,0.000000,27145011200,70352.78987,Stearns County,569,1690,13,...,39,11,13,34,0,0,25,12,0,9.46
1,1,25,21,1133.436461,27145011301,61727.08520,Stearns County,674,1434,108,...,39,11,13,34,0,0,25,12,0,9.46
2,2,3,3,64.505776,27145011302,71496.88658,Stearns County,854,1459,31,...,39,11,13,34,0,0,25,12,0,9.46
3,3,0,0,0.000000,27145011304,86840.15275,Stearns County,640,1116,68,...,39,11,13,34,0,0,25,12,0,9.46
4,4,5,5,164.583303,27145011400,89135.31560,Stearns County,654,1314,15,...,39,11,13,34,0,0,25,12,0,9.46


In [3]:
solar_df.shape

(72537, 169)